# read csv and api key logic

In [ ]:
## solution to missing api key

In [ ]:
<table>
    <thead>
        <tr>
            <th>ID</th>
            <th>API Name</th>
            <th>Category</th>
            <th>Base URL</th>
            <th>Documentation</th>
        </tr>
    </thead>
    <tbody>
        <?php foreach ($results as $row) : ?>
            <tr>
                <td><?= htmlspecialchars($row['id']); ?></td>
                <td><?= htmlspecialchars($row['api_name']); ?></td>
                <td><?= htmlspecialchars($row['category']); ?></td>
                <td><a href="<?= htmlspecialchars($row['base_url']); ?>" target="_blank"><?= htmlspecialchars($row['base_url']); ?></a></td>
                <td><a href="<?= htmlspecialchars($row['docs_url']); ?>" target="_blank">Docs</a></td>
            </tr>
        <?php endforeach; ?>
    </tbody>
</table>


In [ ]:
<?php
// Define the path to your CSV file
$csvFile = 'structured_match_results.csv';

// Initialize an array to hold the CSV data
$csvData = [];

// Open the CSV file for reading
if (($handle = fopen($csvFile, 'r')) !== FALSE) {
    // Read only the first 3 rows
    $count = 0;
    while (($row = fgetcsv($handle, 1000, ",")) !== FALSE && $count < 3) {
        $csvData[] = $row;
        $count++;
    }
    fclose($handle);
}
?>

	<div style="margin-top:20px; padding:20px; border:1px solid red;">
	<TABLE>
    <tr>
        <td>
            <h2>Fixtures</h2>
            <ul>
                <?php foreach ($csvData as $row) : ?>
                    <li><?= htmlspecialchars($row[0]) ?></li>
                <?php endforeach; ?>
            </ul>
        </td>
    </tr>
	</TABLE>

## read_csv.php

In [ ]:
<?php
/**
 * Reads a CSV file with match data, groups rows by the date (YYYY-MM-DD),
 * then separates them into upcoming fixtures (date >= today) and past results (date < today).
 * Returns only a single group for fixtures (the earliest upcoming date) and
 * a single group for results (the most recent past date).
 *
 * @param string $csvFile Path to the CSV file.
 * @return array An associative array with keys:
 *               - 'header': The CSV header row (array).
 *               - 'fixtures': An associative array with one key (the next fixture date) and its rows.
 *               - 'results': An associative array with one key (the most recent past date) and its rows.
 */
function getMatchGroupsFromCsv($csvFile)
{
    $dataByDate = [];
    $header = [];

    if (($handle = fopen($csvFile, "r")) !== FALSE) {
        // Read the header row.
        $header = fgetcsv($handle, 1000, ',');
        
        // Read each row and group by date (using the first 10 characters, YYYY-MM-DD).
        while (($row = fgetcsv($handle, 1000, ',')) !== FALSE) {
            $fullDate = $row[0];                // e.g., "2025-05-11T11:00:00Z"
            $dateOnly = substr($fullDate, 0, 10); // e.g., "2025-05-11"
            $dataByDate[$dateOnly][] = $row;
        }
        fclose($handle);
    }

    $today = date('Y-m-d');
    $resultsByDate = [];  // Past matches: date < today
    $fixturesByDate = []; // Upcoming matches: date >= today

    foreach ($dataByDate as $date => $rows) {
        if ($date < $today) {
            $resultsByDate[$date] = $rows;
        } else {
            $fixturesByDate[$date] = $rows;
        }
    }

    // For past results, sort descending so the most recent date comes first.
    krsort($resultsByDate);
    $mostRecentResult = [];
    if (!empty($resultsByDate)) {
        $resultDates = array_keys($resultsByDate);
        $mostRecentDate = $resultDates[0];
        $mostRecentResult[$mostRecentDate] = $resultsByDate[$mostRecentDate];
    }

    // For fixtures, sort ascending so the earliest upcoming date comes first.
    ksort($fixturesByDate);
    $nextFixture = [];
    if (!empty($fixturesByDate)) {
        $fixtureDates = array_keys($fixturesByDate);
        $nextFixtureDate = $fixtureDates[0];
        $nextFixture[$nextFixtureDate] = $fixturesByDate[$nextFixtureDate];
    }

    return [
        'header'   => $header,
        'fixtures' => $nextFixture,
        'results'  => $mostRecentResult,
    ];
}
?>


## results_print.php

This file includes read_csv.php, calls the function, and then builds a responsive table that only prints the chosen (single) fixture group and the single results group.

In [ ]:
<?php
require_once 'read_csv.php';  // Adjust the path if needed.
$csvFile = 'match_results.csv'; // Path to your CSV file.
$matchData = getMatchGroupsFromCsv($csvFile);

// Retrieve the header.
$header = $matchData['header'];

// Retrieve the single fixture group, if available.
$fixtureKey = !empty($matchData['fixtures']) ? array_key_first($matchData['fixtures']) : null;
$fixtureRows = $fixtureKey ? $matchData['fixtures'][$fixtureKey] : [];

// Retrieve the single result group, if available.
$resultKey = !empty($matchData['results']) ? array_key_first($matchData['results']) : null;
$resultRows = $resultKey ? $matchData['results'][$resultKey] : [];
?>
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Fixtures &amp; Results</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
  <style>
    /* Use fixed layout and allow wrapping in table cells */
    .table {
      table-layout: fixed;
      width: 100%;
    }
    .table th,
    .table td {
      white-space: normal;
      word-wrap: break-word;
    }
  </style>
</head>
<body>
  <div class="container">
    <!-- Upcoming Fixture: Only the next (earliest) fixture group -->
    <h2 class="mt-4">Upcoming Fixture</h2>
    <?php if ($fixtureKey): ?>
      <div class="table-responsive">
        <table class="table table-condensed">
          <thead>
            <tr>
              <th colspan="<?= count($header) ?>">Date: <?= htmlspecialchars($fixtureKey) ?></th>
            </tr>
            <tr>
              <?php foreach ($header as $col): ?>
                <th><?= htmlspecialchars($col) ?></th>
              <?php endforeach; ?>
            </tr>
          </thead>
          <tbody>
            <?php foreach ($fixtureRows as $row): ?>
              <tr>
                <?php foreach ($row as $cell): ?>
                  <td><?= htmlspecialchars($cell) ?></td>
                <?php endforeach; ?>
              </tr>
            <?php endforeach; ?>
          </tbody>
        </table>
      </div>
    <?php else: ?>
      <p>No upcoming fixtures.</p>
    <?php endif; ?>

    <!-- Most Recent Results: Only the most recent past match group -->
    <h2 class="mt-4">Most Recent Results</h2>
    <?php if ($resultKey): ?>
      <div class="table-responsive">
        <table class="table table-condensed">
          <thead>
            <tr>
              <th colspan="<?= count($header) ?>">Date: <?= htmlspecialchars($resultKey) ?></th>
            </tr>
            <tr>
              <?php foreach ($header as $col): ?>
                <th><?= htmlspecialchars($col) ?></th>
              <?php endforeach; ?>
            </tr>
          </thead>
          <tbody>
            <?php foreach ($resultRows as $row): ?>
              <tr>
                <?php foreach ($row as $cell): ?>
                  <td><?= htmlspecialchars($cell) ?></td>
                <?php endforeach; ?>
              </tr>
            <?php endforeach; ?>
          </tbody>
        </table>
      </div>
    <?php else: ?>
      <p>No past results.</p>
    <?php endif; ?>
  </div>
  <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>


## Explanation

CSV Reading and Grouping (read_csv.php):

The function reads the CSV and uses the first 10 characters of the date (YYYY‑MM‑DD) to group rows.

It then splits the groups into fixtures (dates ≥ today) and results (dates < today).

Using ksort and krsort, it orders the fixture groups in ascending order (next fixture first) and the result groups in descending order (most recent past date first).

Finally, it returns an array containing only the first fixture group and the first (most recent) results group.

Displaying Data (results_print.php):

The file includes read_csv.php and calls getMatchGroupsFromCsv().

It uses PHP’s array_key_first() to grab the key from the fixtures and results arrays—ensuring that only one group (per category) is shown.

The table is built using Bootstrap classes to ensure responsiveness and proper styling.

Only one header row (with the unique date) is printed for fixtures and one for results.